# List all countries

In [1]:
paris_agreement_signatories = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", 
    "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas", 
    "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", 
    "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", 
    "Brunei Darussalam", "Bulgaria", "Burkina Faso", "Burundi", "Cabo Verde", 
    "Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad", "Chile", 
    "China", "Colombia", "Comoros", "Congo", "Cook Islands", "Costa Rica", 
    "Croatia", "Cuba", "Cyprus", "Czech Republic", "Democratic People's Republic of Korea", 
    "Democratic Republic of the Congo", "Denmark", "Djibouti", "Dominica", 
    "Dominican Republic", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", 
    "Eritrea", "Estonia", "Eswatini", "Ethiopia", "European Union", "Fiji", "Finland", 
    "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", 
    "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hungary", 
    "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", 
    "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "Kuwait", 
    "Kyrgyzstan", "Lao People's Democratic Republic", "Latvia", "Lebanon", "Lesotho", 
    "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar", 
    "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", 
    "Mauritius", "Mexico", "Micronesia (Federated States of)", "Monaco", "Mongolia", 
    "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia", "Nauru", "Nepal", 
    "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Macedonia", 
    "Norway", "Oman", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", 
    "Peru", "Philippines", "Poland", "Portugal", "Qatar", "Republic of Korea", 
    "Republic of Moldova", "Romania", "Russian Federation", "Rwanda", "Saint Kitts and Nevis", 
    "Saint Lucia", "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", 
    "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", 
    "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", 
    "Spain", "Sri Lanka", "Sudan", "Suriname", "Sweden", "Switzerland", "Syrian Arab Republic", 
    "Tajikistan", "Tanzania", "Thailand", "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago", 
    "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", 
    "United Kingdom of Great Britain and Northern Ireland", "United States of America", "Uruguay", 
    "Uzbekistan", "Vanuatu", "Venezuela", "Viet Nam", "Yemen", "Zambia", "Zimbabwe"
]


# Import and test Tavily Search 

In [2]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=10)

retriever.invoke("Has UK reduced their climate pledge?")

[Document(page_content='Related Topics\nTop Stories\nThe price of peace between Israelis and Palestinians\n\'Absolutely no\': Israel\'s ambassador to UK rejects two-state solution\nPutin tells Russia his war objectives are unchanged\nFeatures\n\'She was like a tsunami\' - the unstoppable rise of Bonmati\nThe risks to Biden as Republicans ramp up impeachment\nHong Kong\'s universities shrivel in Beijing\'s grip\nThe Afghans stuck on the border trying to reach the US\n\'We would vote for peace in Congo - if we had a vote\'\nHamas support grows among Palestinians in West Bank\n The number of public charging points had increased to over 50,000 by the end of October 2023 - up by 45% over the past year - but their deployment requires "rapid acceleration" to meet the increasing demand, the CCC says.\n The miracle drugs priced out of patients\' reach\nCan you solve GCHQ\'s codebreaker challenge for kids?\nWhy 2024 may be the year of \'surprise travel\'\nElsewhere on the BBC\nWhy US \'YOLO\' sp

# Prepare the internet search chain

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

class Response(BaseModel):
    response: str = Field(description="Yes or No on any reduction of the climate pledge")
    policy_delay: str = Field(description="if yes due to Policy delay describe here otherwise N/A")
    scope_change: str = Field(description="if yes due to Scope Change describe here otherwise N/A")
    financing_reduction: str = Field(description="if yes due to financing reduction describe here otherwise N/A")
    other: str = Field(description="if yes due to other reasons describe here otherwise N/A")
    source: str = Field(description="source of the response otherwise N/A")

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}
"""
)

parser = PydanticOutputParser(pydantic_object=Response)

prompt = PromptTemplate(
    template="""Answer the question based only on the context provided.

    {format_instructions}

    Context: {context}

    Question: {question}
    """,
    input_variables=["question", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | llm
    | parser
)

# Run the chain loop over all of the countries

In [5]:
import pandas as pd
from tqdm import tqdm
from time import sleep
df_dict = {"country":[], "response": [], "policy_delay": [], "scope_change": [], "financing_reduction": [], "other": [], "source": []}
for country in tqdm(paris_agreement_signatories):
    df_dict["country"].append(country)
    sleep(0.5)
    try:
        r = chain.invoke({"question": f"Since January 2023, Has {country} announced or implemented any chaanges to its climate or green policy goals, including but not limited to delays in policy implementation, changes in the scope or scale of environmental policies, reduction in financing for green initiatives, or any actions that signify a shift towards increased fossil fuel usage or other environmentally harmful practices? Include details on specific policies affected (e.g. renewable energy targets, emissions reduction goals), nature of the changes (policy delays, scope change, financing reduction) and any notable shifts towards 'more brown' activities like new fossil fuel exploration or drilling licenses."})#({"question": f"Has {country} revised climate goals or green policies since january 2023? include policy delays, scope changes, financing cuts or increased fossil fuel activities."}) #({"question": f"Has {country} reduced their climate pledge"})
        df_dict["response"].append(r.response)
        df_dict["policy_delay"].append(r.policy_delay)
        df_dict["scope_change"].append(r.scope_change)
        df_dict["financing_reduction"].append(r.financing_reduction)
        df_dict["other"].append(r.other)
        df_dict["source"].append(r.source)
    except:
        df_dict["response"].append('There was a problem generating a response')
        df_dict["policy_delay"].append('There was a problem generating a response')
        df_dict["scope_change"].append('There was a problem generating a response')
        df_dict["financing_reduction"].append('There was a problem generating a response')
        df_dict["other"].append('There was a problem generating a response')
        df_dict["source"].append('There was a problem generating a response')
df = pd.DataFrame.from_dict(df_dict)
df.to_csv("anabana_complex.csv") # pick your desired file name

100%|██████████| 194/194 [02:07<00:00,  1.53it/s]
